In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, Imputer

In [2]:
df = pd.read_csv("../data/heloc_dataset_v1.csv")

In [3]:
df.head()

,RiskPerformance,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,Bad,55,144,4,84,20,3,0,83,2,...,43,0,0,0,33,-8,8,1,1,69
1,Bad,61,58,15,41,2,4,4,100,-7,...,67,0,0,0,0,-8,0,-8,-8,0
2,Bad,67,66,5,24,9,0,0,100,-7,...,44,0,4,4,53,66,4,2,1,86
3,Bad,66,169,1,73,28,1,1,93,76,...,57,0,5,4,72,83,6,4,3,91
4,Bad,81,333,27,132,12,0,0,100,-7,...,25,0,1,1,51,89,3,1,0,80


In [4]:
categorical_columns = ["MaxDelqEver", "MaxDelq2PublicRecLast12M"]
continuous_columns = [c for c in df.columns if c not in ["RiskPerformance"] + categorical_columns]

# Remove Rows with No Data
* -9, -8 -7 indicates missing values

In [5]:
df = df.replace([-9], np.nan)


In [6]:
# Remove all -9 

df = df.dropna()

In [7]:
df = df.replace([-8, -7], np.nan)

In [8]:
# Impute -8 and -7 Missing Values

imputer = Imputer(strategy = 'median')
imputed_df = imputer.fit_transform(df[continuous_columns].values)


# One Hot Encode Categorical

In [9]:
categorical_df = pd.get_dummies(df[categorical_columns], columns=categorical_columns)

# Normalize

In [10]:
target_df = df["RiskPerformance"]
target = np.array(target_df == "Good")



In [11]:
scaler = StandardScaler()
continuous_features_scaled = scaler.fit_transform(imputed_df)

# Save as CSV

In [12]:
continuous_df = pd.DataFrame(continuous_features_scaled, columns=continuous_columns, index = df.index)

In [13]:
categorical_df.shape

(9861, 16)

In [14]:
categorical_df.index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            10449, 10450, 10451, 10452, 10453, 10454, 10455, 10456, 10457,
            10458],
           dtype='int64', length=9861)

In [15]:
preprocessed_df = pd.concat([categorical_df, continuous_df], axis=1, join_axes=[categorical_df.index])

In [16]:
preprocessed_df["target"] = target

In [17]:
preprocessed_df.to_csv("preprocessed-fico.csv")